In [1]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py
sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3 import *
from shutil import copyfile

plt.rcParams["figure.figsize"] = (16,12)
plt.rcParams["font.size"] = 24
plt.rcParams['xtick.labelsize'] = "small"
plt.rcParams['ytick.labelsize'] = 36

In [2]:
def repeat(arr, count):
    z_sig = []
    for i in range(count):
        z_sig.extend(arr)
    return z_sig

def mock_injection_noise(alpha_scale,lambda_val,height_sep=-15.5,x_sep=13.9,offset_y=+4.9,phase_shift="TF"):
    stroke=-202
    frequency=3
    axis="z"
    yuk_or_grav="yuk"
    alpha=1
    bead_size=7.6
    signal_sim = force_vs_time(x_sep,height_sep,stroke,frequency,axis,lambda_val,offset_y,yuk_or_grav,alpha,bead_size)
    
    # shift phase of signal according to TF
        # data from Nadav's google doc [https://docs.google.com/spreadsheets/d/132wBYgWQt44M5gDtw48kFwvLDv416yS83ISfGujQUt4/edit#gid=0]

    # convert to bits, this works only for one second chunk!
        # data from Nadav's google doc [https://docs.google.com/spreadsheets/d/132wBYgWQt44M5gDtw48kFwvLDv416yS83ISfGujQUt4/edit#gid=0]

    TF_phases = [1.22,0.97,0.65,0.49,0.3,0.12,0.29,0.13,0.23,0.22,0.27,0.27,0.07]    
    if(phase_shift=="TF"):TF_phase_shift = TF_phases
    elif(phase_shift=="Flip"):TF_phase_shift = Tm = [-1*element for element in TF_phases] 
    elif(phase_shift=="Random_TF"):TF_phase_shift = [1*np.random.normal(element,element/20) for element in TF_phases]
    elif(phase_shift=="Random"):TF_phase_shift =  np.random.uniform(-np.pi,np.pi,len(TF_phases))
    else:TF_phase_shift = len(TF_phases)*[0]
    print(TF_phase_shift)
    harmonics = np.arange(3,len(TF_phases)*3+3,3)

    #print(len(TF_phase_shift))
    # 6 charges new
    amplitude_scale = [2.40E-17,1.35E-17,1.05E-17,9.78E-18,9.45E-18,9.81E-18,9.82E-18,1.04E-17,9.92E-18,7.50E-18,8.37E-18,7.10E-18,7.84E-18]

    fft_sim = np.fft.rfft(signal_sim[1]) 
    new_fft_sim = fft_sim.copy()
    new_fft_shift_only = fft_sim.copy()
    new_fft_sim[0]=1/np.mean(amplitude_scale) * np.abs(fft_sim[0])
    for harmonic in harmonics:
        new_fft_shift_only[harmonic] = np.abs(fft_sim[harmonic])*np.exp(1j*(np.angle(fft_sim[harmonic])+TF_phase_shift[harmonic//3-1]))
        new_fft_sim[harmonic] = 1/amplitude_scale[harmonic//3-1] * np.abs(fft_sim[harmonic])*np.exp(1j*(np.angle(fft_sim[harmonic])+TF_phase_shift[harmonic//3-1]))

    signal_sim_shift=np.fft.irfft(new_fft_shift_only) 
    signal_sim_shift_scale =np.fft.irfft(new_fft_sim) 
    
    
    # extend to 10s
    z_sig = repeat(signal_sim_shift_scale,10)
    # scale to desired alpha
    z_trans_amp = np.multiply(z_sig,alpha_scale)
    
    return z_trans_amp,TF_phase_shift,amplitude_scale

In [3]:
def file_creator(in_file_paths,out_dir,direction,run_no,max_file=1000):
    i=0
    out_dirs=  out_dir+"/%d/" %run_no
    try: os.makedirs(out_dirs)
    except: print("Did not create %s. It may exist or you do not have perimissions." %out_dir)    

    for file_path in in_file_paths:
        i+=1
        if(i<(max_file+1)):
            #if (i % 10==0):print("%f percent" %(i/max_file*100))
            with h5py.File(file_path, 'r') as f_in:
                quad_data_in = np.array(f_in['quad_data'])
                data_in = quad_data_in.reshape(-1,12).T[9]
                #plt.plot(np.array(data_in))
                if(direction=="push"): z_sn = np.subtract(np.array(data_in),np.array(z_trans_amp))
                elif(direction=="pull"): z_sn = np.add(np.array(data_in),np.array(z_trans_amp))
                else: print("Force has to go either way")
                quad_data_in.reshape(-1,12).T[9] = z_sn
                f_in.close()
                #plt.plot(np.array(z_sn),color="black")
            out_path = out_dirs + "Noise_batch_%d.h5" %(i-1)
            copyfile(src=file_path, dst=out_path)
            with h5py.File(out_path, 'r+') as f_out:
                del f_out['quad_data']
                f_out.create_dataset("quad_data", quad_data_in.shape,data=quad_data_in)
                f_out.close()
    return print("Done")

In [4]:
def metafile_creator(x):
    f = open(out_dir+"run_%d.txt" %x, "w")
    f.write("Run number: %d\n alpha value: %f \n lambda value: %f \n direction: %s \n %s %s \n Amplitude scale %s" %(run_no[x],alpha_scale_in[x],lambda_val_in[x],direction_in[x],phase_shift_in[x],str(TF_phase_shift),str(amplitude_scale)))
    f.close()
    return

In [5]:
in_dir = r"/data/new_trap_processed/mockfiles/20200320/raw/noise/"
out_dir = r"/data/new_trap_processed/mockfiles/20200320/output/noise/testrun/"

in_file_paths = []
for file_path in glob.iglob(in_dir+"*.h5"):
    in_file_paths.append(file_path)



z_trans_amp,TF_phase_shift,amplitude_scale = mock_injection_noise(alpha_scale=1e8,lambda_val=12,height_sep=-15.5,x_sep=13.9,offset_y=+4.9,phase_shift="Random")
file_creator(in_file_paths,out_dir,direction="push",run_no=0,max_file=300)

In [6]:
# make lists
alpha_scale_in = []+[np.random.randint(80,800)*1e6 for x in range(25)]+ \
                [np.random.randint(8,80)*1e6 for x in range(25)]+\
                [np.random.randint(8,80)*1e5 for x in range(25)]+\
                8*[1e8]+\
                [np.random.randint(8,800)*1e6 for x in range(16)]
lambda_val_in = [20]+15*[18]+5*[10]+3*[5]+2*[31]+15*[18]+5*[10]+3*[5]+2*[31]+15*[18]+5*[10]+3*[5]+2*[31]+24*[12]
direction_in = 92*["pull"]+8*["push"]
phase_shift_in = 76*["TF"]+4*["Random_TF"]+4*["Random"]+8*["Flip"]+8*["TF"]

In [7]:
run_no =np.arange(0,100,1)
np.random.shuffle(run_no)


for i in tqdm(range(1)):
    print(i, run_no[i])
    z_trans_amp,TF_phase_shift,amplitude_scale = mock_injection_noise(alpha_scale=alpha_scale_in[i],lambda_val=lambda_val_in[i],height_sep=-15.5,x_sep=13.9,offset_y=+4.9,phase_shift=phase_shift_in[i])
    file_creator(in_file_paths,out_dir,direction=direction_in[i],run_no=run_no[i],max_file=1000)
    metafile_creator(i)
np.save(out_dir+"run_no.npy",run_no)
np.save(out_dir+"alpha_scale_in.npy",alpha_scale_in)
np.save(out_dir+"lambda_val_in.npy",lambda_val_in)
np.save(out_dir+"direction_in.npy",direction_in)
np.save(out_dir+"phase_shift_in.npy",phase_shift_in)


  0%|          | 0/1 [00:00<?, ?it/s]

0 11
Loading Gravity Data... Done!
68 2.04837466632e-05
Loaded Yukawa Force
[1.22, 0.97, 0.65, 0.49, 0.3, 0.12, 0.29, 0.13, 0.23, 0.22, 0.27, 0.27, 0.07]


100%|██████████| 1/1 [00:14<00:00, 14.96s/it]

Done
